In [2]:
import os
from langchain_huggingface import ChatHuggingFace
from langchain_core.output_parsers import StrOutputParser
from brain import network
import warnings

from langchain_core.tools import tool

In [3]:
parser = StrOutputParser()


llm = network.llm_selection("meta-llama/Meta-Llama-3-8B-Instruct")

model = ChatHuggingFace(llm=llm, verbose=True)
#chain = model | parser

# print(chain.invoke("Hello"))
print(parser)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\KEVAL\.cache\huggingface\token
Login successful



# pdf reader llm


In [6]:

from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

def pdf_summarize(query: str):
    try:
    
        file_path = (
        f"D:\My-projects\LLM\Dream_Game\{query}.pdf"
        )
        loader = PyMuPDFLoader(file_path)
        docs = loader.load_and_split()

        text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000, chunk_overlap=200, add_start_index=True
            )
        all_splits = text_splitter.split_documents(docs)

        chain = load_summarize_chain(llm, chain_type="stuff",verbose=False)

        result = chain.invoke(all_splits)

        return result["output_text"]
    except Exception as E:
        return str(E)
    
   


pdf = StructuredTool.from_function(
    func=pdf_summarize,
    name="Pdf wizard",
    description="useful for when you need to summarize a given pdf",
    # coroutine= ... <- you can specify an async method if desired as well
)


tools =[pdf]

print(pdf_summarize("Keval_Resume_24"))

In [6]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage,HumanMessage

# prompt = ChatPromptTemplate.from_messages(
#     [
#         (
#             "system",
#             "You are a helpful assistant. Answer all questions to the best of your ability.",
#         ),
#         MessagesPlaceholder(variable_name="messages"),
#     ]
# )

# chain = prompt | model

In [1]:
history = [HumanMessage(content="hi! I'm bob"),AIMessage(content="Hello bob!")]
while True:
    user_query = input()
    if 'exit' in user_query:
        break

    history.extend([HumanMessage(content=user_query)])
    response = model.invoke(history)
    print(response.content)
    history.extend([AIMessage(content=response.content)])
    
    

NameError: name 'HumanMessage' is not defined

In [ ]:
# from langchain.agents import create_tool_calling_agent
# from langchain.agents import initialize_agent

# agent = initialize_agent(tools,
#                          llm,
#                          agent="zero-shot-react-description",
#                          verbose=True)

# # agent.agent.llm_chain.prompt.template = 


# print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Pdf wizard(query: str) - useful for when you need to summarize a given pdf

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Pdf wizard]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
